In [1]:
import pandas as pd
import numpy as np
import math as m

In [2]:
data=pd.read_csv('data.csv')
data_T=pd.DataFrame(data, columns= ['Object','Property','Value','Source'])
data_T

,Object,Property,Value,Source
0,Object1,Property1,179475,Source1
1,Object1,Property3,371893,Source1
2,Object2,Property1,422854,Source1
3,Object2,Property3,324512,Source1
4,Object3,Property1,489481,Source1
...,...,...,...,...
145,Object6,Property2,319997,Source4
146,Object7,Property2,269456,Source4
147,Object8,Property2,323152,Source4
148,Object9,Property2,286530,Source4


In [3]:
data_T['TSInitial']=0.8
data_T['C_v']=0.0
data_T.head(5)

,Object,Property,Value,Source,TSInitial,C_v
0,Object1,Property1,179475,Source1,0.8,0.0
1,Object1,Property3,371893,Source1,0.8,0.0
2,Object2,Property1,422854,Source1,0.8,0.0
3,Object2,Property3,324512,Source1,0.8,0.0
4,Object3,Property1,489481,Source1,0.8,0.0


In [4]:
# Definition de la matice de similarite
def similarity(dataframe):
    dict_sim = {}
    for key, df in dataframe.groupby(by=['Object','Property']):
        Values = df['Value'].unique()
        row = key[0]+key[1]
        for  i in range(len(Values)):
            w1 = Values[i]
            for  u in range(len(Values)):
                w2 = Values[u]
                sim = 1
                if w1!=w2:
                    t = abs(w1-w2)
                    sim = 1/t
                dict_sim[row+str(w1)+str(w2)] = sim
                dict_sim[row+str(w2)+str(w1)] = sim
    return dict_sim

In [5]:
import math
def Ts(df):
    ts_list = df['TSInitial'].to_list()
    return sum([-math.log(1-ts) for ts in ts_list])
def cos_sim(a,b):
    return float(np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b)))

In [6]:
import time as t
from natsort import index_natsorted, order_by_index
def TruthFinder_seq(data_T):
    ni=5
    trustold = 0.00001
    start = t.time()
    nb_iter_do = 0
    rho=0.7
    lam=0.5
    dict_sim=similarity(data_T)
    for i in range(ni):
        df = data_T[['Source', 'TSInitial']].drop_duplicates().reindex(index=order_by_index(data_T[['Source', 'TSInitial']].drop_duplicates().index, index_natsorted(data_T[['Source', 'TSInitial']].drop_duplicates()['Source'], reverse=False)))
        ts1=df['TSInitial']
        grouped_df = data_T.groupby(["Object", "Property","Value"])
        for key,item in grouped_df:
            sum_ = Ts(item)
            data_T.loc[((data_T["Object"] == key[0]) & (data_T["Property"]==key[1]) & (data_T["Value"]==key[2])), "C_v"]=sum_
        grouped_df=data_T.groupby(["Object", "Property","Value"])
        for key ,item in grouped_df:
            cv=data_T.loc[((data_T["Object"] == key[0]) & (data_T["Property"]==key[1]) & (data_T["Value"]==key[2])), "C_v"].values[0]
#             print(cv)

            data_d=data_T.loc[((data_T["Object"] == key[0]) & (data_T["Property"]==key[1]) & (data_T["Value"]!=key[2])), ["Value","C_v"]]
            for row in data_d.values:
                Key_sim=key[0]+key[1]+str(key[2])+str(int(row[0]))
                cv+=rho*(row[1]*dict_sim[Key_sim])
            cv=1/(1+np.exp(-lam*cv))
        #     print(cv)
            data_T.loc[((data_T["Object"] == key[0]) & (data_T["Property"]==key[1]) & (data_T["Value"]==key[2])), "C_v"]=cv
        groupe_ts=data_T.groupby(['Source'])
        for key,item in groupe_ts:
            ts_=item['C_v'].mean()
            data_T.loc[((data_T["Source"] == key)), "TSInitial"]=ts_
        df = data_T[['Source', 'TSInitial']].drop_duplicates().reindex(index=order_by_index(data_T[['Source', 'TSInitial']].drop_duplicates().index, index_natsorted(data_T[['Source', 'TSInitial']].drop_duplicates()['Source'], reverse=False)))
        ts2=df['TSInitial']
        print('Itération : ',i+1)
        print(1-abs(cos_sim(ts1,ts2)))
        nb_iter_do = i+1
        if 1-abs(cos_sim(ts1,ts2)) < trustold:
            break
    end = t.time()

    return data_T,end-start,nb_iter_do



In [28]:
result_=TruthFinder_seq(data_T)

Itération :  1
6.089191327829369e-07


In [29]:
result_

(       Object   Property   Value   Source  TSInitial       C_v
 0     Object1  Property1  179475  Source1   0.812253  0.927042
 1     Object1  Property3  371893  Source1   0.812253  0.927042
 2     Object2  Property1  422854  Source1   0.812253  0.927043
 3     Object2  Property3  324512  Source1   0.812253  0.927042
 4     Object3  Property1  489481  Source1   0.812253  0.927042
 ..        ...        ...     ...      ...        ...       ...
 145   Object6  Property2  319997  Source4   0.671172  0.753253
 146   Object7  Property2  269456  Source4   0.671172  0.753252
 147   Object8  Property2  323152  Source4   0.671172  0.753252
 148   Object9  Property2  286530  Source4   0.671172  0.753252
 149  Object10  Property2  856218  Source4   0.671172  0.635995
 
 [150 rows x 6 columns],
 0.5012810230255127,
 1)

In [9]:
def res(result):
    gr=result.groupby(["Object", "Property"])
    data_out={'Object':[],'Property':[],'Value':[]}
    for key,item in gr:
#         print(item.values)
        cv_max=0
        value_pred=None
        for line in item.values:
#             print(line[5])
            if cv_max<line[5]:
                cv_max=line[5]
                value_pred=line[2]
        data_out['Object'].append(key[0])
        data_out['Property'].append(key[1])
        data_out['Value'].append(value_pred)
    return pd.DataFrame(data_out)
        
        

In [11]:
res_=res(result_[0])
# len(res_)

In [12]:
list_ = res_.to_numpy()

In [13]:
data_truth=pd.read_csv('data_truth.csv')
data_truth_=pd.DataFrame(data_truth, columns= ['Object','Property','Value'])


In [27]:
list__=data_T.to_numpy()
# list__

In [20]:
list_1=data_truth_.to_numpy()


In [21]:
def function(list_A,list_B):
    binary=[]
    search_list = [tuple(li[:-3]) for li in list_B]
    for ser_item in search_list:
        if ser_item in list_A:
            binary.append(1)
        else:
            binary.append(0)
    return binary

In [26]:
res = function(list_, list__)
res_truth=function(list_1, list__)
res
# res_truth

[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1]

In [30]:
!pip install scikit-learn
import sklearn

In [31]:
from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

In [32]:
precision_score(res_truth,res)

1.0

In [33]:
accuracy_score(res_truth,res)

1.0

In [35]:
recall_score(res_truth,res)

1.0

In [36]:
f1_score(res_truth,res)

1.0